In [ ]:
import os
import sys
import time
import json
import shutil
import codecs
import requests
import psycopg2
import feedparser
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

sys.path.append("../")

from utils.sql_utils import *
from utils.scrape_utils import *
from utils.data_science_utils import *

In [ ]:
loadRoutine(Verbose.DEBUG)

In [ ]:
pipelineNYTHeadlines(fetchSource=True, verbose=Verbose.INFO)

In [ ]:
pipelineNYTNewsWire(fetchSource=True, verbose=Verbose.INFO) # startPage = 0

In [ ]:
pipelineAPINewsHeadline(fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
all_sources = pd.read_sql("select * from newsdb.sources", getEngine())
all_sources.shape[0]

In [ ]:
all_articles = pd.read_sql_table("articles", getEngine())
all_articles.shape[0]

In [ ]:
all_article_contents = pd.read_sql_table("article_contents", getEngine())
all_article_contents.shape[0]

In [ ]:
sections = getNYTSections(source.api_key)

In [ ]:
createNlpModel(attributes=['title', 'description'])

In [ ]:
getSimilarArticlesFromText('VP pick', 'title', 10)

In [ ]:
getSimilarArticlesFromText('VP pick', 'description', 10)

In [ ]:
getSimilarArticlesFromText('Lebanon minister resigns in wake of deadly Beirut blast', 'title', 10)

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = "https://www.bbc.com/news/live/world-53775682"

In [ ]:
page = requests.get(url)

In [ ]:
page

In [ ]:
print(page.text)

In [ ]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
title = soup.find('h1').contents

In [ ]:
title

In [ ]:
npr_feed_url = "https://feeds.npr.org/%d/rss.xml"

In [ ]:
for i in range(200, 50000):
    data = feedparser.parse(npr_feed_url % i)
    if 'feed' in data and 'title' in data['feed']:
        print("(\"" + (npr_feed_url % i) + "\", \"" + data['feed']['title'].replace(" : NPR", '') + "\"),")